In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/2022-23_pbp.csv")
pbp2023["next_home_play"]=pbp2023.HOMEDESCRIPTION.shift(-1)
pbp2023["next_visitor_play"]=pbp2023.VISITORDESCRIPTION.shift(-1)

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2022-23",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2022-23",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
305,Kevin Love,0.762562
374,Moritz Wagner,0.551020
179,Isaiah Joe,0.457143
411,Patrick Beverley,0.438376
101,Davion Mitchell,0.397790
174,Herbert Jones,0.383505
204,Jalen Brunson,0.381662
480,Tim Hardaway Jr.,0.368317
26,Austin Reaves,0.367606
75,Clint Capela,0.338346


Shoutout to Jaylin Williams, who played in only 60% of games, but drew 1.615 charges per 36 minutes!

# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
415,Paul Reed,1.440000
464,T.J. McConnell,1.436453
468,Tari Eason,1.402817
171,Hamidou Diallo,1.152809
222,Jarred Vanderbilt,1.131429
475,Thaddeus Young,1.126531
257,Jordan Goodwin,1.115789
514,Wenyen Gabriel,1.089474
131,Donovan Mitchell,1.071910
452,Shai Gilgeous-Alexander,1.044507


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
11,Alex Caruso,5.233613
475,Thaddeus Young,4.800000
354,Matisse Thybulle,4.526946
174,Herbert Jones,4.181443
464,T.J. McConnell,4.007882
107,De'Anthony Melton,3.982979
171,Hamidou Diallo,3.822472
394,O.G. Anunoby,3.731092
414,Paul George,3.693642
153,Fred VanVleet,3.668108


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
308,Kevon Looney,14.415126
399,Onyeka Okongwu,14.089655
129,Domantas Sabonis,13.996552
136,Drew Eubanks,13.459296
436,Rudy Gobert,13.293204
75,Clint Capela,13.127820
32,Bismack Biyombo,12.725000
140,Dwight Powell,12.674227
242,Jock Landale,12.638298
205,Jalen Duren,12.519184


Shoutout to Steven Adams, who played in only 51% of games, but generated 16.83 points by screen assists per 36 minutes!

# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [12]:
misses=pbp2023[pbp2023.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
15,Brook Lopez,22
52,Drew Eubanks,18
84,Jaren Jackson Jr.,17
149,Myles Turner,17
190,Walker Kessler,15
31,Daniel Gafford,13
86,Jarrett Allen,13
97,Joel Embiid,13
123,Kristaps Porzingis,13
155,Nick Richards,13


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [13]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
0,Aaron Gordon,11
80,Ivica Zubac,11
202,Tari Eason,10
99,Jarrett Allen,8
111,Joel Embiid,8
139,Kristaps Porzingis,8
182,Patrick Williams,8
220,Walker Kessler,8
8,Anthony Davis,7
121,Julius Randle,7


# The No Time to Taunt Award (presented by Tim Duncan)*

highest percent of blocks that stayed inbounds & recovered by blocker's team (credit to gibberisle for the idea)

In [9]:
blocks=pbp2023[pbp2023.HOMEDESCRIPTION.str.contains('BLOCK',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
blocks['home_or_visitor_block']=np.where(blocks.HOMEDESCRIPTION.str.contains('BLOCK',na=False),"home","visitor")

team_rebounds=pbp2023[pbp2023.HOMEDESCRIPTION.str.endswith('Rebound',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.endswith('Rebound',na=False)]

outer=blocks.merge(team_rebounds,how='outer',indicator=True,on=['GAME_ID','PERIOD','PCTIMESTRING'])
blocks_wo_team_rebs = outer[(outer._merge=='left_only')].drop('_merge', axis=1).drop(list(outer.filter(regex = '_y')), axis = 1)

/Users/sumitro/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [10]:
def recover_after_block(df):
    if ((df.home_or_visitor_block=="home") & (pd.isnull(df.next_visitor_play_x))):
        return 'recovered by own team'
    elif ((df.home_or_visitor_block=="visitor") & (pd.isnull(df.next_home_play_x))):
        return 'recovered by own team'
    else:
        return 'recovered by other team'

In [11]:
blocks_wo_team_rebs['recovered']=blocks_wo_team_rebs.apply(recover_after_block,axis=1)

player_blocks=blocks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'blocks'})

player_blocks_wo_team_rebs=pd.DataFrame(
    blocks_wo_team_rebs.groupby(['PLAYER3_NAME_x','recovered'])
    .size().reset_index().pivot_table(index='PLAYER3_NAME_x',columns='recovered',fill_value=0).to_records()
).rename(columns={"(0, 'recovered by other team')":'blocks_recovered_by_opp',
                 "(0, 'recovered by own team')":'blocks_recovered_by_own'})
player_blk_info=player_blocks.merge(player_blocks_wo_team_rebs,left_on=['PLAYER3_NAME'],right_on='PLAYER3_NAME_x')
player_blk_info["percent_blk_recovered_by_own"]=player_blk_info["blocks_recovered_by_own"]/player_blk_info["blocks"]
player_blk_info.query("blocks>=40").nlargest(10,columns='percent_blk_recovered_by_own',keep='all')

,PLAYER3_NAME,blocks,PLAYER3_NAME_x,blocks_recovered_by_opp,blocks_recovered_by_own,percent_blk_recovered_by_own
172,Jabari Smith Jr.,68,Jabari Smith Jr.,11,49,0.720588
327,Mo Bamba,42,Mo Bamba,6,30,0.714286
63,Christian Wood,67,Christian Wood,12,46,0.686567
349,O.G. Anunoby,46,O.G. Anunoby,4,31,0.673913
187,Jalen Smith,55,Jalen Smith,12,37,0.672727
396,Scottie Barnes,58,Scottie Barnes,16,39,0.672414
120,Draymond Green,54,Draymond Green,9,36,0.666667
301,Luke Kornet,45,Luke Kornet,11,30,0.666667
158,Isaiah Hartenstein,62,Isaiah Hartenstein,13,41,0.661290
365,Patrick Williams,62,Patrick Williams,13,41,0.661290


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [14]:
#get alley oops
alley_oops=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,p1,p2,count
218,Clint Capela,Trae Young,85
774,Onyeka Okongwu,Trae Young,34
595,John Collins,Trae Young,32
372,Dwight Powell,Luka Doncic,29
125,Brandon Clarke,Ja Morant,25
12,Aaron Gordon,Nikola Jokic,22
267,Darius Garland,Evan Mobley,20
269,Darius Garland,Jarrett Allen,19
681,Kyle Anderson,Rudy Gobert,19
38,Andrew Nembhard,Isaiah Jackson,18


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [15]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [16]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [17]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
144,Eric Gordon,0.179710,0.008696,0.011594,0.397101,0.066667,0.069565,0.733333
81,Collin Sexton,0.472464,0.202899,0.107246,0.168116,0.023188,0.060870,1.034783
206,Jalen Green,0.179710,0.075362,0.472464,0.136232,0.005797,0.240580,1.110145
339,Malaki Branham,0.179710,0.217391,0.260870,0.078261,0.011594,0.475362,1.223188
313,Klay Thompson,0.179710,0.339130,0.104348,0.269565,0.075362,0.315942,1.284058
424,Reggie Bullock,0.179710,0.130435,0.026087,0.246377,0.486957,0.260870,1.330435
450,Seth Curry,0.179710,0.040580,0.188406,0.639130,0.110145,0.213043,1.371014
231,Jaylen Nowell,0.179710,0.115942,0.057971,0.692754,0.107246,0.266667,1.420290
134,Doug McDermott,0.179710,0.382609,0.159420,0.002899,0.292754,0.434783,1.452174
90,Damion Lee,0.179710,0.121739,0.498551,0.043478,0.434783,0.213043,1.491304


In [18]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
411,Patrick Beverley,0.979710,0.208696,0.269565,0.744928,0.144928,0.353623,2.701449
438,Russell Westbrook,0.721739,0.176812,0.379710,0.663768,0.246377,0.750725,2.939130


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [19]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
415,Paul Reed,0.672464,0.959420,0.905797,0.953623,0.892754,0.988406,5.372464
475,Thaddeus Young,0.942029,0.713043,0.811594,0.991304,0.686957,0.872464,5.017391
174,Herbert Jones,0.973913,0.614493,0.892754,0.985507,0.597101,0.898551,4.962319
11,Alex Caruso,0.959420,0.582609,0.991304,0.997101,0.478261,0.944928,4.953623
17,Andre Drummond,0.713043,0.846377,0.663768,0.930435,0.898551,0.852174,4.904348
232,Jaylin Williams,1.000000,0.881159,0.689855,0.408696,0.991304,0.791304,4.762319
135,Draymond Green,0.898551,0.857971,0.936232,0.872464,0.901449,0.172464,4.639130
505,Usman Garuba,0.776812,0.817391,0.988406,0.779710,0.672464,0.582609,4.617391
182,Isaiah Roby,0.852174,0.762319,0.889855,0.620290,0.576812,0.913043,4.614493
374,Moritz Wagner,0.985507,0.736232,0.457971,0.704348,0.895652,0.814493,4.594203


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [21]:
charges=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
98,Giannis Antetokounmpo,22
198,Kyle Kuzma,19
172,Julius Randle,15
12,Anthony Edwards,14
236,O.G. Anunoby,14
16,Bam Adebayo,13
18,Bennedict Mathurin,13
66,Deni Avdija,13
241,Paolo Banchero,13
148,Jerami Grant,12


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [22]:
goaltends=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
36,Daniel Gafford,18
164,Nic Claxton,18
165,Nick Richards,16
83,Jalen Duren,13
147,Mark Williams,12
90,Jaren Jackson Jr.,11
160,Myles Turner,11
185,Rudy Gobert,11
13,Bam Adebayo,10
33,Clint Capela,10


# "The Thing about Arsenal Is They Always Try to Walk It In" Award (presented by MLS Commissioner Don Garber)*

most kicked ball violations

In [20]:
kicked_balls=pbp2023[pbp2023.HOMEDESCRIPTION.str.contains('Kick',na=False)|
                     pbp2023.VISITORDESCRIPTION.str.contains('Kick',na=False)]
kicked_balls.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
240,Nikola Jokic,47
236,Nic Claxton,19
241,Nikola Vucevic,19
129,Jakob Poeltl,17
79,Domantas Sabonis,15
126,Jaden McDaniels,13
175,Jusuf Nurkic,13
63,De'Aaron Fox,11
203,Luka Doncic,11
82,Dorian Finney-Smith,10


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked with at least a 5-inch difference in height

In [28]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
75,Fred VanVleet,27
203,Patrick Beverley,22
164,Kyrie Irving,20
33,Chris Paul,17
66,Donovan Mitchell,16
24,CJ McCollum,15
163,Kyle Lowry,15
43,Damian Lillard,12
52,De'Anthony Melton,11
134,Josh Okogie,11


# The Most 3-Pt Shooting Fouls Committed Award*

credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea

In [23]:
third_free_throws=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
86,Dillon Brooks,10
71,De'Anthony Melton,9
77,Deni Avdija,9
145,Jalen McDaniels,9
222,Luguentz Dort,9
50,Coby White,8
70,De'Andre Hunter,8
100,Evan Mobley,8
312,Trae Young,8
323,Tyrese Maxey,8


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [24]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
34,Patty Mills,0.000
121,Tyus Jones,0.069
93,Seth Curry,0.083
76,Dennis Schroder,0.089
63,Austin Rivers,0.096
131,Cameron Payne,0.096
15,Eric Gordon,0.101
251,Trae Young,0.110
331,Tyrese Haliburton,0.112
340,Tyrese Maxey,0.113


alternatively: restricting to players > 6 foot 6 inches in height

In [25]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
75,Joe Ingles,0.131
149,Terrence Ross,0.154
3,Amir Coffey,0.170
11,Bojan Bogdanovic,0.176
67,Jayson Tatum,0.181
42,Gordon Hayward,0.186
132,Paul George,0.187
40,Georges Niang,0.188
93,Kevin Huerter,0.190
18,Cedi Osman,0.192


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [26]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
261,Wenyen Gabriel,0.645
207,Luke Kornet,0.593
241,Mitchell Robinson,0.592
290,Romeo Langford,0.578
402,Isaiah Jackson,0.555
157,Jakob Poeltl,0.547
57,Anthony Davis,0.546
406,Day'Ron Sharpe,0.543
486,Christian Koloko,0.535
31,Taj Gibson,0.527


alternatively: restricting to players < 6 foot 7 inches in height

In [27]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
149,Romeo Langford,0.578
49,Derrick Jones Jr.,0.521
100,Josh Okogie,0.469
108,Kenrich Williams,0.425
3,Aaron Wiggins,0.410
78,Ish Wainright,0.375
58,Draymond Green,0.366
118,Luguentz Dort,0.365
9,Anthony Lamb,0.352
69,Grant Williams,0.341


# The Rotation Awards

I value depth over one solitary star (it’s a rotation after all). My attempt to make this more objective than past years:
- Get position estimates from Basketball-Reference
    - players with most minutes at PG & SG: guard
    - players with most minutes at SG & SF or SF & PF: wing
    - players with most minutes at PC & C: big
    - exceptions: all players greater than 7 foot are bigs and all players less than 6 foot are guards
- Filter out players with less than 12 MP/G
- Rank teams based on highest positional VORP without positional VORP leader

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

East
1.	Wizards (Bradley Beal, Delon Wright, Monte Morris, Jordan Goodwin, Kendrick Nunn)
2.	76ers (James Harden, De’Anthony Melton, Tyrese Maxey)
3.	Cavaliers (Donovan Mitchell, Darius Garland, Ricky Rubio)
4.	Boston (Derrick White, Malcolm Brogdon, Marcus Smart, Payton Pritchard)
5.	Hawks (Trae Young, Dejounte Murray, Trent Forrest, Aaron Holiday)

West
1.	Mavericks (Luka Doncic, Spencer Dinwiddie, Kyrie Irving)
2.	Grizzlies (Ja Morant, Tyus Jones, Luke Kennard)
3.	Warriors (Steph Curry, Donte DiVincenzo, Ty Jerome, Jordan Poole, Gary Payton II)
4.  Nuggets (Jamal Murray, Bruce Brown, Bones Hyland, Reggie Jackson)
5.	Pelicans (CJ McCollum, Devonte’ Graham, Jose Alvarado, Dyson Daniels)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

East
1.	Boston (Jayson Tatum, Jaylen Brown, Sam Hauser, Grant Williams)
2.	Bulls (DeMar DeRozan, Zach LaVine, Derrick Jones Jr, Ayo Dosunmu)
3.	Raptors (Scottie Barnes, Chris Boucher, OG Anunoby, Thad Young, Juancho Hernangomez)
4.	Cavaliers (Caris LeVert, Isaac Okoro, Dean Wade, Cedi Osman, Lamar Stevens)
5.	Pistons (Bojan Bogdanovic, Alec Burks, Saddiq Bey, Hamidou Diallo, Kevin Knox, Eugene Omoruyi, Isaiah Livers, Rodney McGruder)

West
1.	Clippers (Kawhi Leonard, Paul George, Terance Mann, Robert Covington, Marcus Morris, Luke Kennard, Amir Coffey)
2.	Suns (Devin Booker, Mikal Bridges, Cam Johnson, Torrey Craig, Ish Wainright, Damion Lee, Terrence Ross, TJ Warren)
3.	Kings (Kevin Huerter, Harrison Barnes, Keegan Murray, Malik Monk, Terence Davis, Kessler Edwards)
4.	Pelicans (Trey Murphy III, Brandon Ingram, Herb Jones, Josh Richardson, Naji Marshall)
5.	Timberwolves (Anthony Edwards, Kyle Anderson, Jaden McDaniels, Nickeil Alexander-Walker, Taurean Prince, Jaylen Nowell)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

East
1.	Bucks (Giannis Antetokounmpo, Brook Lopez, Bobby Portis)
2.	Nets (Kevin Durant, Nic Claxton, Cam Johnson)
3.	Cavaliers (Jarrett Allen, Evan Mobley, Kevin Love)
4.	Knicks (Julius Randle, Mitchell Robinson, Isaiah Hartenstein, Jericho Sims, Obi Toppin)
5.	Magic (Wendell Carter Jr, Mo Wagner, Mo Bamba, Bol Bol, Goga Bitadze)

West
1.	Lakers (LeBron James, Anthony Davis, Thomas Bryant, Jarred Vanderbilt, Wenyen Gabriel)
2.  Grizzlies (Jaren Jackson Jr, Brandon Clarke, Santi Aldama, Steven Adams, Xavier Tillman, Jake LaRavia)	
3.	Jazz (Lauri Markkanen, Walker Kessler, Jarred Vanderbilt, Kelly Olynyk, Damian Jones, Rudy Gay)
4.	Pelicans (Zion Williamson, Larry Nance Jr, Jonas Valanciunas, Willy Hernangomez, Jaxson Hayes)
5.  Timberwolves (Rudy Gobert, Naz Reid, Karl-Anthony Towns)

Teams with no representation in any Rotation Awards: Heat, Pacers, Hornets, Thunder, Trail Blazers, Spurs, Rockets